@misc{lee2024nvembed,
      title={NV-Embed: Improved Techniques for Training LLMs as Generalist Embedding Models}, 
      author={Chankyu Lee and Rajarshi Roy and Mengyao Xu and Jonathan Raiman and Mohammad Shoeybi and Bryan Catanzaro and Wei Ping},
      year={2024},
      eprint={2405.17428},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
}


https://huggingface.co/nvidia/NV-Embed-v1

In [2]:
import os
os.chdir('../')
os.getcwd()

'/Users/dennis/repos/study/mt/mt_rdf'

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('data/raw/ppaths_join_nl.csv')

In [5]:
access_token=os.environ.get('HUGGINGFACE_TOKEN')

In [6]:
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from huggingface_hub import login

In [7]:
login(token=access_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/dennis/.cache/huggingface/token
Login successful


In [8]:
# load model with tokenizer
model = AutoModel.from_pretrained('nvidia/NV-Embed-v1', trust_remote_code=True) #, use_auth_token=True, token=access_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
task_name_to_instruct = {"example": "Given a Natural Language Question based on a SPARQL Query, return '1' if the original SPARQL Query has a faster runtime if you turn off the virtuoso optmizer or '0' otherwise.",}

In [10]:
df

,query_id,query,label
0,P180,Which municipal arrondissements are located in...,0
1,P181,What is the field of occupation of Stéphane Du...,1
2,P182,Which administrative territorial entities are ...,0
3,P183,What is the ISO 3166-2 code of the administrat...,0
4,P184,What is the ISO 3166-2 code of the administrat...,0
...,...,...,...
209,P399,Which paintings created by a member of the Roy...,0
210,P400,What are the locations and types of mountains ...,0
211,P401,What are the different types of musical instru...,0
212,P402,In which administrative territorial entities d...,0


In [11]:
df.iloc[1]['query']

'What is the field of occupation of Stéphane Dujarric?'

In [12]:
query_prefix = "Instruct: "+task_name_to_instruct["example"]+"\nQuery: "

queries = list(df['query'])

In [13]:
max_length = 250
query_embeddings = model.encode(queries, instruction=query_prefix, max_length=max_length)

/Users/dennis/.cache/huggingface/modules/transformers_modules/nvidia/NV-Embed-v1/7f6188488250b5bd3a334d93dfce0f1291f240e3/modeling_nvembed.py:345: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_dict.get('input_ids').to(batch_dict.get('input_ids')).long()),


: 

In [16]:
query_embeddings

tensor([[-0.0145,  0.0103,  0.0050,  ..., -0.0050,  0.0089,  0.0301]])

In [17]:
len(query_embeddings[0])

4096

In [23]:
double_df = df.copy()
double_df = pd.concat([double_df] *2, ignore_index=True)

In [27]:
double_df['label'].sum()

50

In [26]:
df['label'].sum()

25